<a href="https://colab.research.google.com/github/cstrachan342/Alim-Confiance/blob/main/solution/Alim_confiance_write_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dataset Creation**

In this notebook I will detail how I created my Dataset by sourcing and merging auxiliary datasets. By doing so I add contextual information providing additional dimensions to the data, which will enable models to gain an understanding of complex relationships to improve overal predictive performance.


In [1]:
!git clone https://github.com/cstrachan342/Alim-Confiance

Cloning into 'Alim-Confiance'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 43 (delta 8), reused 34 (delta 3), pack-reused 0
Receiving objects: 100% (43/43), 8.72 MiB | 20.15 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [2]:
!pip install -q category_encoders
!pip install -q haversine
!pip install -q unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import polars as pl
import numpy as np
import re
import math
from datetime import datetime
from unidecode import unidecode
import difflib
import haversine as hs
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

The challenge  was not a time series problem as it initially looks like. From my feature engineering below you will see I have treated the dataset **mostly** as a time series problem with aspects that are not time series. This was the most successful way I found to optimise the given metric of accuracy. If I was using the Alim'Confidance dataset for the stereotypical use I would treat it solely as a time series issue and not engineer some of the features I did.  

In [4]:
data = pd.read_csv('/content/Alim-Confiance/data/alim_confiance_dataset/alim_confiance_train.csv')

In [5]:
data

,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Synthese_eval_sanit,Agrement,geores,filtre,ods_type_activite
0,SAVEURS ASIATIQUES,50846842800034,RUE DES FRERES LUMIERE,33130,Bègles,23-110093-1,2024-02-05T01:00:00+01:00,Restaurant,Satisfaisant,NaN,44.797031_-0.535231,Restaurant,Autres
1,EPICERIE ROND POINT,90472398800013,48 AV DU ROND POINT,93250,Villemomble,23-056603-1,2023-07-17T02:00:00+02:00,Libre service|Alimentation générale,Satisfaisant,NaN,48.884745_2.499984,Libre service|Alimentation générale,Autres
2,AUBERGE DES OLIVIERS,48792443300013,ROUTE DE LA CANONICA,20290,Lucciana,23-018114-1,2023-03-08T01:00:00+01:00,Restaurant,Très satisfaisant,NaN,42.541715_9.46286,Restaurant,Autres
3,LES HUITRES DES FLOTS,80481822700022,RUE DU PONT DES BERNES,50550,Saint-Vaast-la-Hougue,23-011747-1,2023-02-15T01:00:00+01:00,Purification/Expédition de coquillages,Satisfaisant,50562016,49.592002_-1.28678,NaN,Produits de la mer et d'eau douce
4,COLONNA DOMINIQUE-ANTOINE,53444598600019,Partinello,20147,Partinello,23-097885-1,2023-11-21T01:00:00+01:00,Producteur fermier,Très satisfaisant,NaN,42.306077_8.67833,Producteur fermier,Autres
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22666,AUX DELICES DE CHARENTONG,88764629700010,28 RUE DE PARIS,94220,Charenton-le-Pont,23-009992-1,2023-02-14T01:00:00+01:00,Restaurant,Satisfaisant,NaN,48.818757_2.417811,Restaurant,Autres
22667,PICARD SURGELES,78493968801468,15 BOULEVARD BRUNE,75014,Paris 14e Arrondissement,23-051981-1,2023-05-26T02:00:00+02:00,Libre service,Très satisfaisant,NaN,48.825397_2.314932,Libre service,Autres
22668,CFA DE HAUTE SAONE,13002607300135,5 RUE DU TALEROT,70000,Vesoul,23-048979-1,2023-06-21T02:00:00+02:00,Découpe de viandes de volailles/lagomorphe,Très satisfaisant,70550005,47.633819_6.155241,NaN,Viandes et produits carnés
22669,LA FABRIQUE,49360317900042,LA FABRIQUE,08000,Charleville-Mézières,23-069730-1,2023-08-16T02:00:00+02:00,Restaurant,Très satisfaisant,NaN,49.772441_4.720325,Restaurant,Autres


In [6]:
data_t = pd.read_csv('/content/Alim-Confiance/data/alim_confiance_dataset/alim_confiance_test.csv')

In the cities dataset it contains a list of cities in France, with departments and regions as well as GPS coordinates distrubuted by Data Gouv FR. Original dataset can be found at https://www.data.gouv.fr/en/datasets/villes-de-france/

In [7]:
city = pd.read_csv('/content/Alim-Confiance/data/cities.csv')

The dataset below contains population numbers and amount of deaths split by age groups. I will be using the most up to date year as possible which is 2019. While I downloaded the dataset from kaggle for simplicity, the original publishing comes from https://github.com/scrouzet/covid19-incrementality

The kaggle dataset can be found at https://www.kaggle.com/datasets/lperez/death-and-population-in-france-19902019

In [8]:
pop = pd.read_csv('/content/Alim-Confiance/data/population_insee_dept_year_sex_age.csv', sep=';')
death = pd.read_csv('/content/Alim-Confiance/data/INSEE_deces_2010_2019.csv', sep=';')

Firms dataset is again taken from Kaggle but originally from National Institute of Statistics and Economic Studies (INSEE). This is the official french institute for gathering data on demographics, economics and more. This specific dataset is preprocessed to contain data on firms and number of employees in these firms per town. Kaggle link is https://www.kaggle.com/datasets/etiennelq/french-employment-by-town


In [9]:
firms = pd.read_csv('/content/Alim-Confiance/data/base_etablissement_par_tranche_effectif.csv', sep=',')

In [10]:
y_map = {'A corriger de manière urgente' : 0.0,
         'A améliorer' : 1.0,
         'Satisfaisant' : 2.0,
         'Très satisfaisant' : 3.0}

In [11]:
data['Synthese_eval_sanit'] = data['Synthese_eval_sanit'].map(y_map)

One of my major breakthroughs in creating my dataset was combining test and train data. This raised my accuracy in to the 67% area early on. It allowed for easy and accurate calculations of aggregate features.  

In [12]:
data['trustii_id'] = 0
data_t['Synthese_eval_sanit'] = 5
data = pd.concat([data, data_t])
data.reset_index(drop=True, inplace=True)

Below are functions that will be used later on the dataframe.

In [13]:
def calculate_distance(x):
        address = data[data['Numero_inspection']==str(x)].Libelle_commune.values
        address = unidecode(address[0].lower()).replace('-', ' ')
        post_code = data[data['Numero_inspection']==str(x)].Code_postal.values
        match_ = difflib.get_close_matches(address, city[city['zip_code'] == post_code[0]].city_code.values, n=1, cutoff=0.1)
        if len(match_) == 0:
            match_ = city[city['city_code']==address]

            if len(match_) >= 1:
              new_address = difflib.get_close_matches(address, match_.city_code.values, n=1, cutoff=0.1)
              if len(new_address) == 0:
                return np.nan
              match_ = match_[match_['city_code']==new_address[0]]

              x_long = data[data['Numero_inspection']==str(x)].Longitude.values[0]
              x_lat = data[data['Numero_inspection']==str(x)].Attitude.values[0]

              city_lat = match_.latitude.values[0]
              city_long = match_.longitude.values[0]

              if math.isnan(x_lat):
                  return np.nan
              elif math.isnan(x_long):
                  return np.nan
              elif math.isnan(city_lat):
                  return np.nan
              elif math.isnan(city_long):
                  return np.nan
              else:
                return hs.haversine((x_long,x_lat),(city_lat,city_long))

            else:
              return np.nan

        else:
            x_long = data[data['Numero_inspection']==str(x)].Longitude.values[0]
            x_lat = data[data['Numero_inspection']==str(x)].Attitude.values[0]

            city_lat = city[city['city_code']==match_[0]].latitude.values[0]
            city_long = city[city['city_code']==match_[0]].longitude.values[0]

            return hs.haversine((x_long,x_lat),(city_lat,city_long))

In [14]:
def calculate_region(x):
        address = data[data['Numero_inspection']==str(x)].Libelle_commune.values
        address = unidecode(address[0].lower()).replace('-', ' ')
        post_code = data[data['Numero_inspection']==str(x)].Code_postal.values
        match_ = difflib.get_close_matches(address, city[city['zip_code'] == post_code[0]].city_code.values, n=1, cutoff=0.1)
        if len(match_) == 0:
            match_ = city[city['city_code']==address]
            if len(match_) >= 1:
                new_reg = difflib.get_close_matches(address, city[city['zip_code'] == post_code[0]].city_code.values, n=1, cutoff=0.1)
                if len(new_reg) == 0:
                  return np.nan
                match_ = city[city['city_code']==address]
                if len(match_) >= 1:
                    return match_.region_name.values[0]
                else:
                    return 'unknown'
        else:
            return city[city['city_code']==match_[0]].region_name.values[0]

In [15]:
def calculate_department(x):
        address = data[data['Numero_inspection']==str(x)].Libelle_commune.values
        address = unidecode(address[0].lower()).replace('-', ' ')
        post_code = data[data['Numero_inspection']==str(x)].Code_postal.values
        match_ = difflib.get_close_matches(address, city[city['zip_code'] == post_code[0]].city_code.values, n=1, cutoff=0.1)
        if len(match_) == 0:
            match_ = city[city['city_code']==address]
            if len(match_) >= 1:
                new_dep = difflib.get_close_matches(address, match_.city_code.values, n=1, cutoff=0.1)
                if len(new_dep) == 0:
                  return np.nan
                match_ = match_[match_['city_code']==new_dep[0]]
                return match_.department_number.values[0]
            else:
              return 'unknown'
        else:
            return city[city['city_code']==match_[0]].department_number.values[0]

In [16]:
def fill_addresses(x):
    bla = data[(data['SIRET']==str(x)) & (data['Adresse_2_UA'].isna())]
    conc = bla['Adresse_2_UA'].astype(str) + '_' + bla['Longitude'].astype(str) + '_' + bla['Attitude'].astype(str)
    conc = conc.drop_duplicates().values
    name = bla['APP_Libelle_etablissement'].astype(str).values[0]
    for i in range(len(conc)):
        data.loc[(data['SIRET']==str(x)) & (data['Adresse_2_UA'].isna()), 'Adresse_2_UA'] = name + '_' + str(i)

Due to the spatial-temporal database the most importance features are going to be related to longitude, lattitude, postcode and date.

In [17]:
data['geores'] = data['geores'].apply(lambda x: x.split('_') if pd.isna(x)==False else [np.nan, np.nan])
data2 = pd.DataFrame(data['geores'].tolist(), columns=['Longitude','Attitude'])
data = pd.concat([data, data2], axis=1, join="inner")
data['Attitude'] = data['Attitude'].astype(np.float32)
data['Longitude'] = data['Longitude'].astype(np.float32)
data['Code_postal'] = data['Code_postal'].apply(lambda x: int(x) if re.search('[a-zA-Z]', x)==None else x)

As there are multiple missing values for longitude and lattitude, I can use my external database to merge on postcode to give a rough location. This will not give the exact location however being in the general viscinity will help the model more than a blank value. I will also need the longitude and lattitude values for feature engineering.

In [18]:
for i in data[data['Longitude'].isna()].Numero_inspection.values:
    if len(city[city['zip_code']==data[data['Numero_inspection']==str(i)].Code_postal.values[0]]) != 0:
        data.loc[data['Numero_inspection']==str(i), 'Longitude'] = city[city['zip_code']==data[data['Numero_inspection']==str(i)].Code_postal.values[0]].latitude.values[0]
        data.loc[data['Numero_inspection']==str(i), 'Attitude'] = city[city['zip_code']==data[data['Numero_inspection']==str(i)].Code_postal.values[0]].longitude.values[0]
    else:
      address = data[data['Numero_inspection']==str(i)].Libelle_commune.values
      address = unidecode(address[0].lower()).replace('-', ' ')
      post_code = data[data['Numero_inspection']==str(i)].Code_postal.values
      match_ = difflib.get_close_matches(address, city[city['zip_code'] == post_code[0]].city_code.values, n=1, cutoff=0.1)
      if len(match_) == 0:
         match_ = city[city['city_code'] == address]
         if len(match_) >= 1:
            match_ = difflib.get_close_matches(address, match_.city_code.values, n=1, cutoff=0.1)
            data.loc[data['Numero_inspection']==str(i), 'Longitude'] = city[city['city_code']==match_[0]].latitude.values[0]
            data.loc[data['Numero_inspection']==str(i), 'Attitude'] = city[city['city_code']==match_[0]].longitude.values[0]

      else:
        data.loc[data['Numero_inspection']==str(i), 'Longitude'] = city[city['zip_code']==match_[0]].longitude.values[0]
        data.loc[data['Numero_inspection']==str(i), 'Attitude'] = city[city['zip_code']==match_[0]].latitude.values[0]

I will fill in blanks for addresses by replacing all blanks with the street address and a number to create unique entries.

In [19]:
data[data['Adresse_2_UA'].isna()].SIRET.drop_duplicates().apply(fill_addresses)
print('Blanks Filled')

Blanks Filled


Due to the importance of location features, mapping existing values of longitude and lattitude into a 3D space will give the model more information and allow it to learn deeper features of the data.

In [20]:
data['x'] = np.cos(data['Attitude']) * np.cos(data['Longitude'])
data['y'] = np.cos(data['Attitude']) * np.sin(data['Longitude'])
data['z'] = np.sin(data['Attitude'])

As the data is time series based a large portion of my code is going to map values based on date. Even though the challenge itself was not time series I found most success with the accuracy when treating the data in this way.

In [21]:
data['prev_insp'] = 0
for i in data['SIRET']:
  dates = data[data['SIRET']==str(i)].Date_inspection
  dates = dates.unique()
  if len(dates) != 1:
    map_dates = {i:v for v, i in enumerate(sorted(dates, reverse=True))}
    data.loc[data['SIRET'] == str(i), 'prev_insp'] = data[data['SIRET']==str(i)]['Date_inspection'].map(map_dates)

Due to the categorical data I will one hot encode the neccesary features. In this scenario you can use either `get_dummies()` or `OneHotEncoder()`. The only benefit to `get_dummies()` is it can separate data when all combined into one column if you specificy the separator in the code.

In [22]:
ohe_type = ce.one_hot.OneHotEncoder(use_cat_names=True)
data3 = ohe_type.fit_transform(data['ods_type_activite'])
data = pd.concat([data, data3], axis=1, join="inner")

data6 = data['filtre'].str.get_dummies(sep='|')
data = pd.concat([data, data6], axis=1, join="inner")

data.loc[data['APP_Libelle_activite_etablissement']=='_', 'APP_Libelle_activite_etablissement'] = 'unknown'
app = data['APP_Libelle_activite_etablissement'].str.get_dummies(sep='|')
app = app.add_prefix('app_')
app_train_columns = app.columns
data = pd.concat([data, app], axis=1, join="inner")

Using Pandas datetime I can manipulate the date of inspection to create multiple features. I can create features for day, month, year and also day of year. Then I can create season and day of the week before encoding both columns.

In [23]:
data['Date_inspection'] = pd.to_datetime((data['Date_inspection'].str.split('T')).str[0], format='%Y-%m-%d')
data['Date_inspection_days'] = data['Date_inspection'].apply(lambda x: (datetime(2024,2,7) - x).days)
data['Date_inspection_days'] = data['Date_inspection_days'].astype(np.float32)

data['year'] = pd.DatetimeIndex(data['Date_inspection']).year
data['month'] = pd.DatetimeIndex(data['Date_inspection']).month
data['day'] = pd.DatetimeIndex(data['Date_inspection']).day
data['dayofyear'] = pd.DatetimeIndex(data['Date_inspection']).dayofyear

data['season'] = pd.cut((data.dayofyear), [0, 80, 172, 266, 355, 366],labels=['winter', 'spring', 'summer', 'autumn', 'winter'], ordered=False)
ohe_season = ce.one_hot.OneHotEncoder(use_cat_names=True)
data_season = ohe_season.fit_transform(data['season'])
data = pd.concat([data, data_season], axis=1, join="inner")

data['dayname'] = pd.DatetimeIndex(data['Date_inspection']).day_name()
ohe_dayname = OrdinalEncoder(categories=[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']])
data_dayname = ohe_dayname.fit_transform(data[['dayname']])
data = pd.concat([data, pd.Series(data_dayname.flatten(), name='daynameenc')], axis=1, join="inner")

I have created a concatatenated column of details in the dataset to create very unique entries which I will use to create new features. The original dataset has multiple inspections over multiple days and mutiple different types of inspections so I can create a column to utilise this information.

In [24]:
data['concat_2'] = data['SIRET'].astype(str) + '_' + data['Date_inspection'].astype(str) + '_' + data['Adresse_2_UA'].astype(str)
data['concat_3'] = data['SIRET'].astype(str) + '_' + data['APP_Libelle_activite_etablissement'].astype(str) + '_' + data['Adresse_2_UA'].astype(str)

As I am treating this as time series I will add the time since the last inspection. The data had multiple entires on the same day with one of the only differences being the `'ods_type_activite'`. I interpretted this as different inspections on different catagories. This led me to add new features not only looking at the SIRET as a whole but to track the unique inspections within it to.

This is why there is a features such as `data['prev_insp_days_unique']` as it looks at the last time this specific type of inspection was done. I do also look at the last time the inspection was done as a whole too as shown by `data['prev_insp_days']`. This is a trend I repeat in the majority of feature engineering I do.

In [25]:
data['prev_insp'] = 0
for i in np.unique(data['SIRET'].values):
  dates = data[data['SIRET']==str(i)].Date_inspection
  dates = dates.unique()
  if len(dates) != 1:
    map_dates = {i:v for v, i in enumerate(sorted(dates, reverse=True))}
    data.loc[data['SIRET'] == str(i), 'prev_insp'] = data[data['SIRET']==str(i)]['Date_inspection'].map(map_dates)

data['prev_insp_days'] = 0
for i in np.unique(data['SIRET'].values):
  dates = data[data['SIRET']==str(i)].Date_inspection_days
  dates = sorted(dates.unique(), reverse=True)
  if len(dates) != 1:
    order = []
    order.append(0)
    for j in range(len(dates)-1):
      order.append(dates[j]-dates[j+1])
    map_dates_days = {v:i for v, i in zip(dates,order)}
    data.loc[data['SIRET'] == str(i), 'prev_insp_days'] = data[data['SIRET']==str(i)]['Date_inspection_days'].map(map_dates_days)

data['prev_insp_days_unique'] = 0
for i in np.unique(data['concat_3'].values):
  dates = data[data['concat_3']==str(i)].Date_inspection_days
  dates = sorted(dates.unique(), reverse=True)
  if len(dates) != 1:
    order = []
    order.append(0)
    for j in range(len(dates)-1):
      order.append(dates[j]-dates[j+1])
    map_dates_days_unique = {v:i for v, i in zip(dates,order)}
    data.loc[data['concat_3']==str(i), 'prev_insp_days_unique'] = data[data['concat_3']==str(i)]['Date_inspection_days'].map(map_dates_days_unique)

Within the data the trend of hygeine scores didnt have much variance between entries, so if it was excelent before it would most likely be excelent for the next inspection. By adding the feature `last known score`, if there has been a previous entry it will return the hygiene score. As I am treating it mostly as a time series dataset it will overlap and always return the last known score dependent on the date.

In [26]:
data['last_known_score'] = 5
for i in np.unique(data['SIRET'].values):
  dates = data[data['SIRET']==str(i)].sort_values(by=['Date_inspection_days']).Date_inspection_days.values
  scores = data[data['SIRET']==str(i)].sort_values(by=['Date_inspection_days']).Synthese_eval_sanit.values
  scores = np.insert(scores, 0, 5)
  if len(dates) != 1:
    map_scores = {v:i for v, i in zip(dates, scores[:-1])}
    data.loc[data['SIRET']==str(i), 'last_known_score'] = data[data['SIRET']==str(i)]['Date_inspection_days'].map(map_scores)

data['last_known_score_unique'] = 5
for i in np.unique(data['concat_3'].values):
  dates = data[data['concat_3']==str(i)].sort_values(by=['Date_inspection_days']).Date_inspection_days.values
  scores = data[data['concat_3']==str(i)].sort_values(by=['Date_inspection_days']).Synthese_eval_sanit.values
  scores = np.insert(scores, 0, 5)
  if len(dates) != 1:
    map_scores_unique = {v:i for v, i in zip(dates, scores[:-1])}
    data.loc[data['concat_3']==str(i), 'last_known_score_unique'] = data[data['concat_3']==str(i)]['Date_inspection_days'].map(map_scores_unique)

One of my biggest breakthroughs in gaining a significant increase in accuracy throughout this competition was when I stopped treating this as **just** a time series task. I realised that within the test set it contains SIRET numbers
with dates in the past. This means it cannot be a time series task as you are required to predict results from entires in the past. So by introducing look-ahead bias I could increase my accuracy score and place on the leaderboard.

While this does not feel like an satisfying aspect to introduce the challenge was not set up in a time series format, so adding this bias help my model in this context. Typically I would not encourage look ahead bias to be exploited.

In [27]:
data['future_insp_days'] = 0
for i in np.unique(data['SIRET'].values):
  dates = data[data['SIRET']==str(i)].Date_inspection_days
  dates = sorted(dates.unique(), reverse=True)
  if len(dates) != 1:
    order = []
    for j in range(len(dates)-1):
      order.append(dates[j]-dates[j+1])
    order.append(0)
    map_dates_days = {v:i for v, i in zip(dates,order)}
    data.loc[data['SIRET'] == str(i), 'future_insp_days'] = data[data['SIRET']==str(i)]['Date_inspection_days'].map(map_dates_days)

data['future_insp_days_unique'] = 0
for i in np.unique(data['concat_3'].values):
  dates = data[data['concat_3']==str(i)].Date_inspection_days
  dates = sorted(dates.unique(), reverse=True)
  if len(dates) != 1:
    order = []
    for j in range(len(dates)-1):
      order.append(dates[j]-dates[j+1])
    order.append(0)
    map_dates_days_unique = {v:i for v, i in zip(dates,order)}
    data.loc[data['concat_3']==str(i), 'future_insp_days_unique'] = data[data['concat_3']==str(i)]['Date_inspection_days'].map(map_dates_days_unique)


By adding future known score below it increases the look-ahead bias and consequently increases overfitting significantly. This is a trade off I made by creating the dataset this way.

In [28]:
data['future_next_score'] = 5
for i in np.unique(data['SIRET'].values):
  dates = data[data['SIRET']==str(i)].sort_values(by=['Date_inspection_days']).Date_inspection_days.values
  scores = data[data['SIRET']==str(i)].sort_values(by=['Date_inspection_days']).Synthese_eval_sanit.values
  scores = np.insert(scores, 0, 5)
  if len(dates) != 1:
    map_scores = {v:i for v, i in zip(dates, sorted(scores[:-1], reverse=True))}
    data.loc[data['SIRET']==str(i), 'future_next_score'] = data[data['SIRET']==str(i)]['Date_inspection_days'].map(map_scores)

data['future_next_score_unique'] = 5
for i in np.unique(data['concat_3'].values):
  dates = data[data['concat_3']==str(i)].sort_values(by=['Date_inspection_days']).Date_inspection_days.values
  scores = data[data['concat_3']==str(i)].sort_values(by=['Date_inspection_days']).Synthese_eval_sanit.values
  scores = np.insert(scores, 0, 5)
  if len(dates) != 1:
    map_scores_unique = {v:i for v, i in zip(dates, sorted(scores[:-1], reverse=True))}
    data.loc[data['concat_3']==str(i), 'future_next_score_unique'] = data[data['concat_3']==str(i)]['Date_inspection_days'].map(map_scores_unique)


Sometimes if there were alot of inspections it could take multiple days, and alot of correlations can be drawn between number of inspections and hygiene score. By value counting my previously concatenated column I can work out the number of inspections done per day.

In [29]:
num_insp = data['concat_2'].value_counts()
num_insp = num_insp.to_frame()
data = data.merge(num_insp,
                  left_on = 'concat_2',
                  right_index = True,
                  how ='left')

data = data.rename(columns={'concat_2_y':'num_insp_per_day', 'concat_2_x':'concat_2'})

Agreement column appeared to be very important. I found best results when  including the number of agreements rather than a binary column indicating presence or absence.

In [30]:
data = pd.concat([data, pd.Series((data['Agrement'].str.get_dummies(sep='|')).sum(axis=1), name='Agrement_conv')], axis=1, join='inner')

By using my earler defined functions `calculate_region` and `calculate_department`, I can use my additional datasets to add in region and departments for all entries. Then using the `calculate_distance` function it measures the distance from city center to the location of the establishment using haversine formula.

In [31]:
data['region'] = data.Numero_inspection.apply(calculate_region)
data['distance'] = data.Numero_inspection.apply(calculate_distance)
data['department'] = data.Numero_inspection.apply(calculate_department)

Now I can encode my categorical features. I use one hot encoder for my region category, however I use ordinal encoder for `SIRET` and `APP_Libelle_etablissement` due to computational reasons, as it would create over 20,000 columns if one hot encoded.

In [32]:
ohe_region = ce.one_hot.OneHotEncoder(use_cat_names=True)
data4 = ohe_region.fit_transform(data['region'])
data = pd.concat([data, data4], axis=1, join="inner")

encoder_title = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value = (len(data.APP_Libelle_etablissement.value_counts())+1))
data9 = encoder_title.fit_transform(data['APP_Libelle_etablissement'].values.reshape(-1, 1))
data = pd.concat([data, pd.Series(data9.flatten(), name='name')], axis=1, join="inner")

encoder_siret = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value = (len(data.SIRET.value_counts())+1))
data10 = encoder_siret.fit_transform(data['SIRET'].values.reshape(-1, 1))
data = pd.concat([data, pd.Series(data10.flatten(), name='siret')], axis=1, join="inner")

Now I can enrich the original dataset with further open source datasets. Below I use my French population dataset to extract further features like the demographic breakdown by gender within different age groups.

In [33]:
pop = pop[pop['annee']==2019]

age_map = {1: 0,
           2: 1}

pop['sexe_code'] = pop['sexe_code'].map(age_map)
pop = pop.drop(columns=['annee', 'sexe'])

pop_homme = pop[pop['sexe_code']==0]
pop_femme = pop[pop['sexe_code']==1]

pop_homme = pop_homme.pivot(index=['departement_code', 'departement', 'sexe_code'],
                            columns='classe_age',
                            values='population').reset_index()
pop_homme['sum_pop'] = pop_homme['00 à 19 ans'] + pop_homme['20 à 39 ans'] + pop_homme['40 à 59 ans'] + pop_homme['60 à 74 ans'] + pop_homme['75 ans et plus']
pop_homme = pop_homme.add_prefix('pop_male_').drop(columns=['pop_male_sexe_code'])

pop_femme = pop_femme.pivot(index=['departement_code', 'departement', 'sexe_code'],
                            columns='classe_age',
                            values='population').reset_index()
pop_femme['sum_pop'] = pop_femme['00 à 19 ans'] + pop_femme['20 à 39 ans'] + pop_femme['40 à 59 ans'] + pop_femme['60 à 74 ans'] + pop_femme['75 ans et plus']
pop_femme = pop_femme.add_prefix('pop_female_').drop(columns=['pop_female_sexe_code'])

data = data.merge(pop_homme, left_on='department', right_on='pop_male_departement_code', how='left')
data = data.merge(pop_femme, left_on='department', right_on='pop_female_departement_code', how='left')
data = data.drop(columns=['pop_male_departement_code', 'pop_female_departement_code', 'pop_female_departement'])

data['pop_00 à 19 ans'] = data['pop_male_00 à 19 ans'] + data['pop_female_00 à 19 ans']
data['pop_20 à 39 ans'] = data['pop_male_20 à 39 ans'] + data['pop_female_20 à 39 ans']
data['pop_40 à 59 ans'] = data['pop_male_40 à 59 ans'] + data['pop_female_40 à 59 ans']
data['pop_60 à 74 ans'] = data['pop_male_60 à 74 ans'] + data['pop_female_60 à 74 ans']
data['pop_75 ans et plus'] = data['pop_male_75 ans et plus'] + data['pop_female_75 ans et plus']
data['pop_sum_pop'] = data['pop_male_sum_pop'] + data['pop_female_sum_pop']

<ipython-input-33-eb08862a54de>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop['sexe_code'] = pop['sexe_code'].map(age_map)


Now using my firms dataset I can merge my data to add new features containing number of firms per city and number of employees present within them. This will indicate the economic growth of the area.

In [34]:
firms = firms.drop(columns=['CODGEO', 'LIBGEO', 'REG']).groupby(by=['DEP'], as_index=False).sum()
data = data.merge(firms, left_on='department', right_on='DEP', how='left')

Next I use a dataset containing numbers of deaths in French population. This dataset allows me to add features containing amount of deaths in French cities split by sex as well as total numbers.

In [35]:
death = death[death['annee_comptabilisation']==2019]
death = death.drop(columns=['annee_comptabilisation', 'date_deces'])
death = death.groupby(by=['sexe','age','departement_deces'], as_index=False).sum()

death['sexe'] = death['sexe'].map(age_map)
death_homme = death[death['sexe']==0]
death_femme = death[death['sexe']==1]

death_homme["age_group"] = pd.cut(x=death_homme['age'], bins=[-1,19,39,49,74,200], labels=['male_00 à 19 ans','male_20 à 39 ans', 'male_40 à 59 ans', 'male_60 à 74 ans', 'male_75 ans et plus'])
death_homme = death_homme.drop(columns=['age', 'sexe']).groupby(by=['departement_deces','age_group'], as_index=False).sum().pivot(index=['departement_deces'],
                                                                                                                                  columns='age_group',
                                                                                                                                  values='nb_deces').reset_index()
death_homme['male_sum_death'] = death_homme['male_00 à 19 ans'] + death_homme['male_20 à 39 ans'] + death_homme['male_40 à 59 ans'] + death_homme['male_60 à 74 ans'] + death_homme['male_75 ans et plus']
death_homme = death_homme.add_prefix('death_')

death_femme["age_group"] = pd.cut(x=death_femme['age'], bins=[-1,19,39,49,74,200], labels=['female_00 à 19 ans','female_20 à 39 ans', 'female_40 à 59 ans', 'female_60 à 74 ans', 'female_75 ans et plus'])
death_femme = death_femme.drop(columns=['age', 'sexe']).groupby(by=['departement_deces','age_group'], as_index=False).sum().pivot(index=['departement_deces'],
                                                                                                                                  columns='age_group',
                                                                                                                                  values='nb_deces').reset_index()
death_femme['female_sum_death'] = death_femme['female_00 à 19 ans'] + death_femme['female_20 à 39 ans'] + death_femme['female_40 à 59 ans'] + death_femme['female_60 à 74 ans'] + death_femme['female_75 ans et plus']
death_femme = death_femme.add_prefix('death_')

data = data.merge(death_homme, left_on='department', right_on='death_departement_deces', how='left')
data = data.merge(death_femme, left_on='department', right_on='death_departement_deces', how='left')

data['death_00 à 19 ans'] = data['death_male_00 à 19 ans'] + data['death_female_00 à 19 ans']
data['death_20 à 39 ans'] = data['death_male_20 à 39 ans'] + data['death_female_20 à 39 ans']
data['death_40 à 59 ans'] = data['death_male_40 à 59 ans'] + data['death_female_40 à 59 ans']
data['death_60 à 74 ans'] = data['death_male_60 à 74 ans'] + data['death_female_60 à 74 ans']
data['death_75 ans et plus'] = data['death_male_75 ans et plus'] + data['death_female_75 ans et plus']
data['death_sum_pop'] = data['death_male_sum_death'] + data['death_female_sum_death']

data.loc[data['pop_male_departement'].isna(), 'pop_male_departement'] = 'unknown'

ohe_dep = ce.one_hot.OneHotEncoder(use_cat_names=True)
dep = ohe_dep.fit_transform(data['pop_male_departement'])
data = pd.concat([data, dep], axis=1, join="inner")

<ipython-input-35-e61d2fa3f689>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  death_homme["age_group"] = pd.cut(x=death_homme['age'], bins=[-1,19,39,49,74,200], labels=['male_00 à 19 ans','male_20 à 39 ans', 'male_40 à 59 ans', 'male_60 à 74 ans', 'male_75 ans et plus'])
<ipython-input-35-e61d2fa3f689>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  death_femme["age_group"] = pd.cut(x=death_femme['age'], bins=[-1,19,39,49,74,200], labels=['female_00 à 19 ans','female_20 à 39 ans', 'female_40 à 59 ans',

I generated new aggregated metrics for average hygiene scores, categorized by region and department. I experimented alot with these features, testing averaging different columns and even adding in standard deviations and variance. I found adding these features made my model overfit a lot so ultimately decided to remove them from my dataset.

In [36]:
region_avg = data[data['Synthese_eval_sanit']!=5].groupby(['region'],as_index=False)['Synthese_eval_sanit'].mean().rename(columns={'Synthese_eval_sanit':'region_avg'})
data = data.merge(region_avg, left_on='region', right_on='region', how='left')
dep_avg = data[data['Synthese_eval_sanit']!=5].groupby(['pop_male_departement'],as_index=False)['Synthese_eval_sanit'].mean().rename(columns={'Synthese_eval_sanit':'dep_avg'})
data = data.merge(dep_avg, left_on='pop_male_departement', right_on='pop_male_departement', how='left')

pcode = data[data['Synthese_eval_sanit']!=5].groupby(['Code_postal'],as_index=False).size()
pcode_values = pcode[pcode['size']>=2].Code_postal.values
post_avg = data[(data['Code_postal'].isin(pcode_values)) & (data['Synthese_eval_sanit']!=5)].groupby(['Code_postal'],as_index=False)['Synthese_eval_sanit'].mean().rename(columns={'Synthese_eval_sanit':'post_avg'})
data = data.merge(post_avg, left_on='Code_postal', right_on='Code_postal', how='left')

To enchance the learning process and provide additional context I will add 3 further datasets from INSEE. The SIRET number was essential to creating a successful model as it gave me access to detailed information on specific business addresses, activity codes, legal forms, and more. Below I use establishments stock file (active and closed), legal stock file and establishment succession links stock file.

### **SIRET Establishments Stock File**

Due to the size of the SIRET dataset, I am using `read_csv()` by Polars to load into memory. If I was to load it using Pandas my session would crash due to the size. Polars efficiently reads large datasets by leveraging a lazy evaluation engine and parallel processing, optimizing performance for big data tasks.

After it is loaded I will filter to only the have relevant data for all SIRET number in my dataset. Then I will convert back to Pandas as I prefer the code syntax and familiarity of the module over Polars.

In [173]:
sir = pl.read_csv('/content/Alim-Confiance/data/StockEtablissement_utf8_SIRET.csv', low_memory=True, infer_schema_length=10000, ignore_errors=True, null_values=['[ND]'])

In [174]:
siret_n =  data[data['SIRET'].str.contains('[a-zA-Z]')==False]
siret_n['SIRET'] = siret_n['SIRET'].astype(int)
siret_n = pl.DataFrame(data = siret_n.SIRET.values, schema=['SIRET'])
siret_n = siret_n.join(sir, left_on='SIRET', right_on='SIRET', how='left')

<ipython-input-174-8c4f9b24c483>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  siret_n['SIRET'] = siret_n['SIRET'].astype(int)


In [175]:
siret_miss =  data[data['SIRET'].str.contains('[a-zA-Z]')==True]
siret_miss = siret_miss.SIRET

In [176]:
siret_n = siret_n.to_pandas()

In [177]:
siret_n = pd.concat([siret_n, pd.DataFrame(siret_miss, columns=siret_n.columns)], axis=0)

In [178]:
siret_n = siret_n.drop_duplicates(subset=['SIRET'])
siret_n.reset_index(drop=True, inplace=True)

From the SIRET Establishments Stock File we are able to gain many addtional features. Most notably Main Establishment Activity Classification, Establishment Creation Date and Establishment Number of Employees.

The features are typically either binary or categorical, so alot of my time was spent setting values to 0/1 or using ```get_dummies()``` to encode the categorical features.



In [179]:
siret_n['activitePrincipaleEtablissement'] = siret_n['activitePrincipaleEtablissement'].fillna('princ_unknown')
princ = siret_n['activitePrincipaleEtablissement'].str.get_dummies(sep='|')
siret_n = pd.concat([siret_n, princ], axis=1, join="inner")

In [180]:
siret_n['activitePrincipaleRegistreMetiersEtablissement'] = siret_n['activitePrincipaleRegistreMetiersEtablissement'].fillna('princ_reg_unknown')
princ_reg = siret_n['activitePrincipaleRegistreMetiersEtablissement'].str.get_dummies(sep='|')
siret_n = pd.concat([siret_n, princ_reg], axis=1, join="inner")

In [181]:
siret_n['caractereEmployeurEtablissement'] = siret_n['caractereEmployeurEtablissement'].fillna('N')
changement_map = {'O':1,
                  'N':0}

siret_n['caractereEmployeurEtablissement'] = siret_n['caractereEmployeurEtablissement'].map(changement_map)

In [182]:
siret_n['codeCommuneEtablissement'] = siret_n['codeCommuneEtablissement'].fillna(0)
encoder_code = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value=np.nan)
code = encoder_code.fit_transform(siret_n['codeCommuneEtablissement'].values.reshape(-1, 1))
siret_n = pd.concat([siret_n, pd.Series(code.flatten(), name='code')], axis=1, join="inner")

In [183]:
siret_n['codePaysEtrangerEtablissement'] = siret_n['codePaysEtrangerEtablissement'].apply(lambda x: 0 if pd.isnull(x) else 1)

In [184]:
siret_n['dateDebut'] = siret_n['dateDebut'].fillna(datetime(2024,2,7))
siret_n['dateDebut'] = pd.to_datetime(siret_n['dateDebut'], format='%Y-%m-%d')
siret_n['dateDebut'] = siret_n['dateDebut'].apply(lambda x: (datetime(2024,2,7) - x).days)

In [185]:
siret_n['dateCreationEtablissement'] = siret_n['dateCreationEtablissement'].fillna(datetime(2024,2,7))
siret_n['dateCreationEtablissement'] = pd.to_datetime(siret_n['dateCreationEtablissement'], format='%Y-%m-%d')
siret_n['dateCreationEtablissement'] = siret_n['dateCreationEtablissement'].apply(lambda x: (datetime(2024,2,7) - x).days)

In [186]:
siret_n['dateDernierTraitementEtablissement'] = siret_n['dateDernierTraitementEtablissement'].str.split('T').str[0]
siret_n['dateDernierTraitementEtablissement'] = siret_n['dateDernierTraitementEtablissement'].fillna(datetime(2024,2,7))
siret_n['dateDernierTraitementEtablissement'] = pd.to_datetime(siret_n['dateDernierTraitementEtablissement'], format='%Y-%m-%d')
siret_n['dateDernierTraitementEtablissement'] = siret_n['dateDernierTraitementEtablissement'].apply(lambda x: (datetime(2024,2,7) - x).days)

In [187]:
siret_n['denominationUsuelleEtablissement'] = siret_n['denominationUsuelleEtablissement'].fillna("'")
encoder_denomination = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value=np.nan)
deno = encoder_denomination.fit_transform(siret_n['denominationUsuelleEtablissement'].values.reshape(-1, 1))
siret_n = pd.concat([siret_n, pd.Series(deno.flatten(), name='deno')], axis=1, join="inner").drop(columns=['denominationUsuelleEtablissement'])

In [188]:
siret_n['distributionSpecialeEtablissement'] = siret_n['distributionSpecialeEtablissement'].fillna('dist_unknown')
siret_n.loc[siret_n['distributionSpecialeEtablissement']=='      ', 'distributionSpecialeEtablissement'] = '0_'
dist = siret_n['distributionSpecialeEtablissement'].str.get_dummies(sep='|')
siret_n = pd.concat([siret_n, dist], axis=1, join="inner")

In [189]:
siret_n['enseigne1Etablissement'] = siret_n['enseigne1Etablissement'].fillna("'")
encoder_etab = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value=np.nan)
etab = encoder_etab.fit_transform(siret_n['enseigne1Etablissement'].values.reshape(-1, 1))
siret_n = pd.concat([siret_n, pd.Series(etab.flatten(), name='etab')], axis=1, join="inner").drop(columns=['enseigne1Etablissement'])

In [190]:
siret_n['etablissementSiege'] = siret_n['etablissementSiege'].fillna(True)
boolean_map = {True:1,
               False:0}

siret_n['etablissementSiege'] = siret_n['etablissementSiege'].map(boolean_map)

In [191]:
siret_n['etatAdministratifEtablissement'] = siret_n['etatAdministratifEtablissement'].fillna('F')
admin_map = {'A':1,
             'F':0}

siret_n['etatAdministratifEtablissement'] = siret_n['etatAdministratifEtablissement'].map(admin_map)

In [192]:
siret_n['indiceRepetitionEtablissement'] = siret_n['indiceRepetitionEtablissement'].fillna('indi_unknown')
siret_n.loc[siret_n['indiceRepetitionEtablissement']==' ', 'indiceRepetitionEtablissement'] = 'indi_unknown'
indi = siret_n['indiceRepetitionEtablissement'].str.get_dummies(sep='|')
siret_n = pd.concat([siret_n, indi], axis=1, join="inner")

In [193]:
siret_n['libelleCedexEtablissement'] = siret_n['libelleCedexEtablissement'].fillna('cedex_unknown')
cedex = siret_n['libelleCedexEtablissement'].str.get_dummies(sep='|')
siret_n = pd.concat([siret_n, cedex], axis=1, join="inner")

In [194]:
siret_n['libelleCommuneEtrangerEtablissement'] = siret_n['libelleCommuneEtrangerEtablissement'].apply(lambda x: 0 if pd.isnull(x) else 1)

In [195]:
siret_n['libellePaysEtrangerEtablissement'] = siret_n['libellePaysEtrangerEtablissement'].apply(lambda x: 0 if pd.isnull(x) else 1)

In [196]:
siret_n['nic'] = siret_n['nic'].fillna(0)

In [197]:
siret_n['nombrePeriodesEtablissement'] = siret_n['nombrePeriodesEtablissement'].fillna(0)

In [198]:
siret_n['nomenclatureActivitePrincipaleEtablissement'] = siret_n['nomenclatureActivitePrincipaleEtablissement'].fillna('NAFRev2')
nomen = siret_n['nomenclatureActivitePrincipaleEtablissement'].str.get_dummies(sep='|')
siret_n = pd.concat([siret_n, nomen], axis=1, join="inner")

In [199]:
siret_n['siren'] = siret_n['siren'].fillna(0)

In [200]:
siret_n['statutDiffusionEtablissement'] = siret_n['statutDiffusionEtablissement'].fillna('O')
diff_map = {'P':1,
            'O':0}

siret_n['statutDiffusionEtablissement'] = siret_n['statutDiffusionEtablissement'].map(diff_map)

In [201]:
siret_n['trancheEffectifsEtablissement'] = siret_n['trancheEffectifsEtablissement'].fillna("UNK")
encoder_tranche = OrdinalEncoder(categories=[['UNK', 'NN', '00', '01', '02', '03', '11', '12', '21', '22', '31', '32', '41', '42', '51', '52', '53']])
tranche = encoder_tranche.fit_transform(siret_n['trancheEffectifsEtablissement'].values.reshape(-1, 1))
siret_n = pd.concat([siret_n, pd.Series(tranche.flatten(), name='tranche')], axis=1, join="inner").drop(columns=['trancheEffectifsEtablissement'])

In [202]:
siret_n['typeVoieEtablissement'] = siret_n['typeVoieEtablissement'].fillna("voie_UNK")
voie = siret_n['typeVoieEtablissement'].str.get_dummies(sep='|')
siret_n = pd.concat([siret_n, voie], axis=1, join="inner")

Once all features are encoded I will remove the remaining categorical features that are irrelevant to my task.

In [203]:
siren_dtypes = pd.DataFrame(siret_n.dtypes)
drop_cols = siren_dtypes[siren_dtypes[0]=='object'].index

In [204]:
na_vals = pd.DataFrame(siret_n.isna().sum())
add_drop_cols = na_vals[na_vals[0]!=0].index
drop_cols = np.append(drop_cols, add_drop_cols)
drop_cols = np.delete(drop_cols, 0)

In [205]:
siret_n = siret_n.drop(columns=drop_cols)

As some column names are shared over the SIRET files, I have added a prefix to differentiate them.

In [206]:
siret_n = siret_n.add_prefix('n_').rename(columns={'n_SIRET':'SIRET', 'n_siren':'siren', 'n_nic':'nic'})

### **SIRET Legal Stock File**

Again due to the size of the files I will be using polars then converting to pandas once necesssary.

In [207]:
leg = pl.read_csv('/content/Alim-Confiance/data/StockUniteLegale_utf8_SIRET.csv', low_memory=True, infer_schema_length=10000, ignore_errors=True, null_values=['[ND]'])

In [208]:
siret_l =  siret_n[['siren']]
siret_l = pl.DataFrame(data = np.unique(siret_n.siren.values), schema=['siren'])
siret_l = siret_l.with_columns(pl.col('siren').cast(pl.Int64))
siret_l = siret_l.join(leg, left_on='siren', right_on='siren', how='left')

In [209]:
siret_l = siret_l.to_pandas()

The main features I am able to extract from the SIRET legal stock file are the Main Activity of the Legal Entity, Legal Entity Creation Date and Legal Entity Workforce Employee Count. Again the overwhelming majority are either categorical or binary features, so I convert them where necessary below.

In [210]:
siret_l['caractereEmployeurUniteLegale'] = siret_l['caractereEmployeurUniteLegale'].fillna('O')
unite_map = {'O':1,
             'N':0}

siret_l['caractereEmployeurUniteLegale'] = siret_l['caractereEmployeurUniteLegale'].map(unite_map)

In [211]:
siret_l['categorieEntreprise'] = siret_l['categorieEntreprise'].fillna('UNK')
entre_map = {'GE':3,
             'ETI':2,
             'PME':1,
             'UNK':0}

siret_l['categorieEntreprise'] = siret_l['categorieEntreprise'].map(entre_map)

In [212]:
siret_l['categorieJuridiqueUniteLegale'] = siret_l['categorieJuridiqueUniteLegale'].fillna(5710.0)
siret_l['categorieJuridiqueUniteLegale'] = siret_l['categorieJuridiqueUniteLegale'].astype('str')
juri = siret_l['categorieJuridiqueUniteLegale'].str.get_dummies(sep='|')
siret_l = pd.concat([siret_l, juri], axis=1, join="inner")

In [213]:
siret_l['dateCreationUniteLegale'] = siret_l['dateCreationUniteLegale'].fillna(datetime(1900,1,1))
siret_l['dateCreationUniteLegale'] = pd.to_datetime(siret_l['dateCreationUniteLegale'], format='%Y-%m-%d')
siret_l['dateCreationUniteLegale'] = siret_l['dateCreationUniteLegale'].apply(lambda x: (datetime(2024,2,7) - x).days)

In [214]:
siret_l['dateDebut'] = siret_l['dateDebut'].fillna(datetime(2024,2,7))
siret_l['dateDebut'] = pd.to_datetime(siret_l['dateDebut'], format='%Y-%m-%d', errors='coerce')
siret_l['dateDebut'] = siret_l['dateDebut'].fillna(datetime(2024,2,7))
siret_l['dateDebut'] = siret_l['dateDebut'].apply(lambda x: (datetime(2024,2,7) - x).days)

In [215]:
siret_l['dateDernierTraitementUniteLegale'] = pd.to_datetime((siret_l['dateDernierTraitementUniteLegale'].str.split('T')).str[0], format='%Y-%m-%d')
siret_l['dateDernierTraitementUniteLegale'] = siret_l['dateDernierTraitementUniteLegale'].fillna(datetime(2024,2,7))
siret_l['dateDernierTraitementUniteLegale'] = pd.to_datetime(siret_l['dateDernierTraitementUniteLegale'], format='%Y-%m-%d', errors='coerce')
siret_l['dateDernierTraitementUniteLegale'] = siret_l['dateDernierTraitementUniteLegale'].apply(lambda x: (datetime(2024,2,7) - x).days)

In [216]:
siret_l['denominationUniteLegale'] = siret_l['denominationUniteLegale'].fillna("'")
encoder_deno_leg = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value=np.nan)
deno_leg = encoder_deno_leg.fit_transform(siret_l['denominationUniteLegale'].values.reshape(-1, 1))
siret_l = pd.concat([siret_l, pd.Series(deno_leg.flatten(), name='deno_leg')], axis=1, join="inner").drop(columns=['denominationUniteLegale'])

In [217]:
siret_l['denominationUsuelle1UniteLegale'] = siret_l['denominationUsuelle1UniteLegale'].fillna("'")
encoder_deno_usue = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value=np.nan)
deno_usue = encoder_deno_usue.fit_transform(siret_l['denominationUsuelle1UniteLegale'].values.reshape(-1, 1))
siret_l = pd.concat([siret_l, pd.Series(deno_usue.flatten(), name='deno_usue')], axis=1, join="inner").drop(columns=['denominationUsuelle1UniteLegale'])

In [218]:
siret_l['economieSocialeSolidaireUniteLegale'] = siret_l['economieSocialeSolidaireUniteLegale'].fillna('UNK')
economie_map = {'O':2,
                'N':1,
                'UNK':0}

siret_l['economieSocialeSolidaireUniteLegale'] = siret_l['economieSocialeSolidaireUniteLegale'].map(economie_map)

In [219]:
siret_l['etatAdministratifUniteLegale'] = siret_l['etatAdministratifUniteLegale'].fillna('A')
admin_leg_map = {'A':1,
                 'C':0}

siret_l['etatAdministratifUniteLegale'] = siret_l['etatAdministratifUniteLegale'].map(admin_leg_map)

In [220]:
siret_l['identifiantAssociationUniteLegale'] = siret_l['identifiantAssociationUniteLegale'].apply(lambda x: 0 if pd.isnull(x) else 1)

In [221]:
siret_l['nicSiegeUniteLegale'] = siret_l['nicSiegeUniteLegale'].fillna(0)

In [222]:
siret_l['nombrePeriodesUniteLegale'] = siret_l['nombrePeriodesUniteLegale'].fillna(0)

In [223]:
siret_l['sexeUniteLegale'] = siret_l['sexeUniteLegale'].fillna('UNK')
sexe_leg_map = {'M':2,
                'F':1,
                'UNK':0}

siret_l['sexeUniteLegale'] = siret_l['sexeUniteLegale'].map(sexe_leg_map)

In [224]:
siret_l['societeMissionUniteLegale'] = siret_l['societeMissionUniteLegale'].fillna('UNK')
miss_leg_map = {'N':2,
                'O':1,
                'UNK':0}

siret_l['societeMissionUniteLegale'] = siret_l['societeMissionUniteLegale'].map(miss_leg_map)

In [225]:
siret_l['statutDiffusionUniteLegale'] = siret_l['statutDiffusionUniteLegale'].fillna('O')
stat_leg_map = {'O':0,
                'P':1}

siret_l['statutDiffusionUniteLegale'] = siret_l['statutDiffusionUniteLegale'].map(stat_leg_map)

In [226]:
siret_l['trancheEffectifsUniteLegale'] = siret_l['trancheEffectifsUniteLegale'].fillna("UNK")
encoder_leg_tranche = OrdinalEncoder(categories=[['UNK', 'NN', '00', '01', '02', '03', '11', '12', '21', '22', '31', '32', '41', '42', '51', '52', '53']])
tranche_leg = encoder_leg_tranche.fit_transform(siret_l['trancheEffectifsUniteLegale'].values.reshape(-1, 1))
siret_l = pd.concat([siret_l, pd.Series(tranche_leg.flatten(), name='tranche_leg')], axis=1, join="inner").drop(columns=['trancheEffectifsUniteLegale'])

In [227]:
dtpyes_siret_l = pd.DataFrame(siret_l.dtypes)
drop_cols = dtpyes_siret_l[dtpyes_siret_l[0]=='object'].index

In [228]:
siret_l = siret_l.drop(columns=np.append(drop_cols, ['anneeCategorieEntreprise', 'anneeEffectifsUniteLegale']))

In [229]:
siret_l = siret_l.add_prefix('l_').rename(columns={'l_siren':'siren'})

### **SIRET Succession Links Stock File**

Again same as other two Stock Files I am able to use polars then pandas.

In [288]:
sire = pl.read_csv('/content/Alim-Confiance/data/StockEtablissementLiensSuccession_utf8_SIRET.csv', low_memory=True, infer_schema_length=10000, ignore_errors=True, null_values=['[ND]'])

In [289]:
siret_s =  data[data['SIRET'].str.contains('[a-zA-Z]')==False]
siret_s['SIRET'] = siret_s['SIRET'].astype(int)
siret_s = pl.DataFrame(data = np.unique(siret_s.SIRET.values), schema=['siret'])
siret_s = siret_s.with_columns(pl.col('siret').cast(pl.Int64))
siret_s = siret_s.join(sire, left_on='siret', right_on='siretEtablissementSuccesseur', how='left')

<ipython-input-289-5808bac76d59>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  siret_s['SIRET'] = siret_s['SIRET'].astype(int)


In [290]:
siret_s = siret_s.to_pandas()

In [291]:
siret_miss =  data[data['SIRET'].str.contains('[a-zA-Z]')==True]
siret_miss = siret_miss.SIRET.rename('siret')

siret_s = pd.concat([siret_s, pd.DataFrame(siret_miss, columns=siret_s.columns)], axis=0)

In [292]:
siret_s = siret_s.drop_duplicates(subset=['siret']).rename(columns={'siret':'SIRET'})
siret_s.reset_index(drop=True, inplace=True)

The main use of this stock file is to find if there predecessor or successor for any SIREN number and the creation date. First I will pre-process predecessor data.

In [293]:
siret_s['dateLienSuccession'] = siret_s['dateLienSuccession'].fillna(datetime(2024,2,7))
siret_s['dateLienSuccession'] = pd.to_datetime(siret_s['dateLienSuccession'], format='%Y-%m-%d', errors='coerce')
siret_s['dateLienSuccession'] = siret_s['dateLienSuccession'].apply(lambda x: (datetime(2024,2,7) - x).days)

In [294]:
siret_s['transfertSiege'] = siret_s['transfertSiege'].fillna('UNK')
trans_suc_map = {True:2,
                 False:1,
                 'UNK':0}

siret_s['transfertSiege'] = siret_s['transfertSiege'].map(trans_suc_map)

In [295]:
siret_s['continuiteEconomique'] = siret_s['continuiteEconomique'].fillna('UNK')
econo_suc_map = {True:2,
                 False:1,
                 'UNK':0}

siret_s['continuiteEconomique'] = siret_s['continuiteEconomique'].map(econo_suc_map)

In [280]:
siret_s['dateDernierTraitementLienSuccession'] = pd.to_datetime((siret_s['dateDernierTraitementLienSuccession'].str.split('T')).str[0], format='%Y-%m-%d')
siret_s['dateDernierTraitementLienSuccession'] = siret_s['dateDernierTraitementLienSuccession'].fillna(datetime(2024,2,7))
siret_s['dateDernierTraitementLienSuccession'] = siret_s['dateDernierTraitementLienSuccession'].apply(lambda x: (datetime(2024,2,7) - x).days)

In [281]:
siret_s['siretEtablissementPredecesseur'] = siret_s['siretEtablissementPredecesseur'].fillna(0)

In [284]:
siret_s['has_pred'] = siret_s['siretEtablissementPredecesseur'].apply(lambda x: 0 if int(x)==0 else 1)

In [285]:
siret_s = siret_s.add_prefix('s_').rename(columns={'s_SIRET':'SIRET'})

In [286]:
siret_s['s_siretEtablissementPredecesseur'] = siret_s['s_siretEtablissementPredecesseur'].astype(str)

Now I can repeat the process for successor data.

In [243]:
siret_s2 =  data[data['SIRET'].str.contains('[a-zA-Z]')==False]
siret_s2['SIRET'] = siret_s2['SIRET'].astype(int)
siret_s2 = pl.DataFrame(data = np.unique(siret_s2.SIRET.values), schema=['siret'])
siret_s2 = siret_s2.with_columns(pl.col('siret').cast(pl.Int64))
siret_s2 = siret_s2.join(sire, left_on='siret', right_on='siretEtablissementPredecesseur', how='left')

<ipython-input-243-b9dfdfa43ec8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  siret_s2['SIRET'] = siret_s2['SIRET'].astype(int)


In [244]:
siret_s2 = siret_s2.to_pandas()

In [245]:
siret_miss =  data[data['SIRET'].str.contains('[a-zA-Z]')==True]
siret_miss = siret_miss.SIRET.rename('siret')

siret_s2 = pd.concat([siret_s2, pd.DataFrame(siret_miss, columns=siret_s2.columns)], axis=0)

In [246]:
siret_s2 = siret_s2.drop_duplicates(subset=['siret']).rename(columns={'siret':'SIRET'})
siret_s2.reset_index(drop=True, inplace=True)

In [247]:
siret_s2['dateLienSuccession'] = siret_s2['dateLienSuccession'].fillna(datetime(2024,2,7))
siret_s2['dateLienSuccession'] = pd.to_datetime(siret_s2['dateLienSuccession'], format='%Y-%m-%d', errors='coerce')
siret_s2['dateLienSuccession'] = siret_s2['dateLienSuccession'].apply(lambda x: (datetime(2024,2,7) - x).days)

In [248]:
siret_s2['transfertSiege'] = siret_s2['transfertSiege'].fillna('UNK')
trans_suc_map = {True:2,
                 False:1,
                 'UNK':0}

siret_s2['transfertSiege'] = siret_s2['transfertSiege'].map(trans_suc_map)

In [249]:
siret_s2['continuiteEconomique'] = siret_s2['continuiteEconomique'].fillna('UNK')
econo_suc_map = {True:2,
                 False:1,
                 'UNK':0}

siret_s2['continuiteEconomique'] = siret_s2['continuiteEconomique'].map(econo_suc_map)

In [250]:
siret_s2['dateDernierTraitementLienSuccession'] = pd.to_datetime((siret_s2['dateDernierTraitementLienSuccession'].str.split('T')).str[0], format='%Y-%m-%d')
siret_s2['dateDernierTraitementLienSuccession'] = siret_s2['dateDernierTraitementLienSuccession'].fillna(datetime(2024,2,7))
siret_s2['dateDernierTraitementLienSuccession'] = siret_s2['dateDernierTraitementLienSuccession'].apply(lambda x: (datetime(2024,2,7) - x).days)

In [251]:
siret_s2['siretEtablissementSuccesseur'] = siret_s2['siretEtablissementSuccesseur'].fillna(0)

In [252]:
siret_s2['has_succ'] = siret_s2['siretEtablissementSuccesseur'].apply(lambda x: 0 if int(x)==0 else 1)

In [253]:
siret_s2 = siret_s2.add_prefix('s2_').rename(columns={'s2_SIRET':'SIRET'})

In [254]:
siret_s2['s2_siretEtablissementSuccesseur'] = siret_s2['s2_siretEtablissementSuccesseur'].astype(str)

Now I am left with two dataframes, which I can merge and where SIRET numbers have a predecessor or successor it will show and give my models more complex relationships to learn.

### **Dataset Combination**

To merge on SIRET number I have to ensure all entries match, so I will convert all numbers to strings and then strip them of blanks.  

In [255]:
data['SIRET'] = data['SIRET'].astype('str')
data['SIRET'] = data['SIRET'].str.strip()
siret_n['SIRET'] = siret_n['SIRET'].astype(str)
siret_n['SIRET'] = siret_n['SIRET'].str.strip()

In [256]:
data['SIRET'] = data['SIRET'].str.lstrip('0')
data_final = data.merge(siret_n, left_on='SIRET', right_on='SIRET', how='left')

All of the below are unnessecary columns that are either not useful or categorical data that needs to be dropped.

In [257]:
data_cols = data.drop(columns=(['department', 'pop_male_departement', 'death_departement_deces_x', 'death_departement_deces_y', 'DEP', 'region', 'concat_2',
                                'concat_3', 'Code_postal', 'APP_Libelle_etablissement', 'Adresse_2_UA', 'Numero_inspection', 'geores', 'Synthese_eval_sanit', 'Agrement',
                                'APP_Libelle_activite_etablissement', 'ods_type_activite', 'filtre', 'Libelle_commune', 'Date_inspection', 'dayname', 'season', 'trustii_id',
                                ])).columns

Now I can start merging my SIRET Stock Files.

In [258]:
data['SIRET'] = data['SIRET'].astype('str')
data['SIRET'] = data['SIRET'].str.strip()
siret_n['SIRET'] = siret_n['SIRET'].astype(str)
siret_n['SIRET'] = siret_n['SIRET'].str.strip()
data['SIRET'] = data['SIRET'].str.lstrip('0')
data_final = data[data_cols].merge(siret_n, left_on='SIRET', right_on='SIRET', how='left')

In [259]:
data_final = data_final.merge(siret_l, left_on='siren', right_on='siren', how='left')

In [260]:
siret_s['SIRET'] = siret_s['SIRET'].astype(str)
siret_s['SIRET'] = siret_s['SIRET'].str.strip()
data_final = data_final.merge(siret_s, left_on='SIRET', right_on='SIRET', how='left')

The best pre-processing method I found was Min Max Scaling.

In [261]:
scaler = preprocessing.MinMaxScaler()
d = scaler.fit_transform(data_final.drop(columns=['SIRET']))
scaled_df = pd.DataFrame(d, columns=data_final.drop(columns=['SIRET']).columns)
scaled_df = pd.concat([scaled_df, data['Synthese_eval_sanit'], data['trustii_id']], axis=1, join="inner")

I experimented with other methods for dealing with blanks however the most successful by far was `IterativeImputer()`. Due to computational needs I cannot run my whole dataset through it. So I will run my data without the additional SIRET datasets then merge back together after blank values have been dealt with.

In [126]:
imputer = IterativeImputer()
X_imputed = imputer.fit_transform(scaled_df[np.delete(data_cols,0)])

/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [262]:
imputed_df = pd.DataFrame(X_imputed, columns=scaled_df[np.delete(data_cols,0)].columns)
imputed_df = pd.concat([scaled_df.drop(columns=imputed_df.columns), imputed_df], axis=1, join="inner")

In [313]:
imputed_df

,siren,nic,n_statutDiffusionEtablissement,n_dateCreationEtablissement,n_dateDernierTraitementEtablissement,n_etablissementSiege,n_nombrePeriodesEtablissement,n_libelleCommuneEtrangerEtablissement,n_codeCommuneEtablissement,n_codePaysEtrangerEtablissement,...,pop_male_departement_Loiret,pop_male_departement_Tarn-et-Garonne,pop_male_departement_Eure-et-Loir,pop_male_departement_Lozère,pop_male_departement_Essonne,pop_male_departement_Haute-Vienne,pop_male_departement_La Réunion,region_avg,dep_avg,post_avg
0,0.509310,0.000371,0.0,0.096452,0.014433,1.0,0.1250,0.0,0.335037,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.698026,0.372693,0.333333
1,0.906221,0.000142,0.0,0.016987,0.086281,1.0,0.1250,0.0,0.943861,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.573311,0.335061,0.590909
2,0.488732,0.000142,0.0,0.132589,0.014433,1.0,0.1875,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.774176,0.783333
3,0.806150,0.000240,0.0,0.062359,0.014433,1.0,0.0625,0.0,0.512732,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.682954,0.655078,0.875000
4,0.535330,0.000207,0.0,0.091044,0.723077,1.0,0.0625,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32383,0.452068,0.009454,0.0,0.131808,0.014433,0.0,0.1250,0.0,0.953607,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.573311,0.570519,0.250000
32384,0.323531,0.000251,0.0,0.306224,0.014433,1.0,0.2500,0.0,0.761705,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.573311,0.566873,0.725434
32385,0.410614,0.000186,0.0,0.199495,0.001586,1.0,0.2500,0.0,0.367284,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.525567,0.452371,0.644360
32386,0.949483,0.000229,0.0,0.002684,0.017764,1.0,0.0625,0.0,0.428118,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.701739,0.596367,0.664062
